In [ ]:
import re
import numpy as np
import pandas as pd


# O Paradigma Funcional

Vamos recapitular quais paradigmas de programação utilizamos até aqui:

**Paradigma Imperativo**
- O programa é composto por um estado e instruções que modificam este estado:

```python
x = 0
for i in range(10):
    x = (x + i)*2
print x
```
- Forma mais simples de programação, é o paradigma das linguagens mais antigas (C, Fortran e COBOL por exemplo).

**Orientação à Objetos (OOP)**
- O estado e as instruções que operam sobre esse estado são parte de um **objeto**:

```python
class Casa:
    def __init__(self):
        self.porta = 'Aberta'
    
    def fechar_porta(self):
        self.porta = 'Fechada'

    def abrir_porta(self):
        self.porta = 'Aberta'
```
- OOP é um paradigma muito comum hoje em dia, presente em linguagens como C++, Java e Python.

**Functional Programming**
- Não existe estado, apenas funções: o programa é uma sequência de aplicação de funções ao input:
```python
def somar_2(x):
    return x + 2

def mult_4(x):
    return x * 4

saida = somar_2(mult_4(somar_2(entrada)))
```
- No paradigma funcional funções são variáveis.
- Paradigma que surgiu com a linguagem LISP, nos anos 70, e hoje em dia sobrevive nas áreas de dados através de linguagens como R, Julia, Python (em parte).

## Funções são variáveis

Mesmo quando consturímos uma função utilizando `def`, o resultado é uma variável com uma função dentro!

In [ ]:
soma_1 = lambda x: x + 1


In [ ]:
soma_1


In [ ]:
def soma_1_c(x):
    return x + 1


In [ ]:
soma_1_c


In [ ]:
soma_1(soma_1_c(10))


Podemos utilizar esse comportamento para criar funções a partir de outras funções:

In [ ]:
somar_n = lambda x, n: x + n


In [ ]:
soma_1 = lambda x: somar_n(x, 1)


In [ ]:
soma_1(10)


## O Conceito de `map`

Já vimos que podemos mapear uma função à uma `Series` do Pandas através do método `.map()`. Agora veremos como fazer a mesma coisa para qualquer iterável.

In [ ]:
lista_exemplo = [10, 12, 34, 23, 2, 6, 7]


In [ ]:
def div_2(x):
    return x / 2


**Como podemos aplicar a função `div_2` acima à `lista_exemplo`?**

In [ ]:
div_2(lista_exemplo)


Não conseguimos aplicar a função diretamente: afinal, `div_2` espera receber 1 argumento inteiro, e `lista_exemplos` é uma lista!

Podemos utilizar programação imperativa e construir um loop:

In [ ]:
new_list = []

for item in lista_exemplo:
    new_list.append(div_2(item))

new_list


Outra forma, já da programação funcional, são os lists comprehensions:

In [ ]:
[div_2(item) for item in lista_exemplo]


Uma terceira forma é através da **função** `map()`:

In [ ]:
for i in map(div_2, lista_exemplo):
    print(i)


O resultado de `map()` é preguiçoso (*lazy*): isso significa que ele não é calculado até ser chamado diretamente. No exemplo acima utilizamos um loop para chamar cada elemento do resultado. Podemos transformar o resultado em uma lista, que forçará o Python a avaliar todas as execuções da função `div_2()`:

In [ ]:
list(map(div_2, lista_exemplo))


Um comportamento curioso dos iteradores preguiçosos é que eles se esvaziam conforme chamamos os seus elementos:

In [ ]:
resultado_map = map(div_2, lista_exemplo)
for i in resultado_map:
    print(i)


In [ ]:
list(resultado_map)


In [ ]:
resultado_map = map(div_2, lista_exemplo)


In [ ]:
list(resultado_map)


### Lazy evaluation

A avaliação lazy é um conceito fundamental em Big Data: ela economiza memória e tempo de processamento realizando avaliações somente quando estas são necessárias.

In [ ]:
lista_telefones = [
    19999571559,
    "(21) 2412-0107",
    "(34) 99762-1166",
    "91-4002-8282",
    "(19) 3542-1820",
    "(19) 3561-9525",
    "(34) 3333-5802",
]
pattern = r"[0-9]{2}"


In [ ]:
lista_dds = list(map(lambda x: re.findall(pattern, str(x))[0], lista_telefones))
print(lista_dds)


In [ ]:
for ddd in map(lambda x: "".join(re.findall(pattern, str(x)))[:2], lista_telefones):
    print(ddd)


## Filtrando iteráveis com `filter()`

A segunda parte importante do paradigma funcional é a função `filter()`: ela nos permite filtrar os elementos de um iterável a partir de uma função que retorna valores booleanos. Assim como `map()`, `filter()` avalia (de forma preguiçosa) um iterável e retorna apenas os elementos onde a função aplicada retorna `True`.

Vamos continuar o nosso exemplo com uma lista de telefones e uma função para extrair o DDD:

In [ ]:
lista_telefones = [
    19999571559,
    "(21) 2412-0107",
    "(34) 99762-1166",
    "91-4002-8282",
    "(19) 3542-1820",
    "(19) 3561-9525",
    "(34) 3333-5802",
]


def extrair_ddd(telefone):
    """
    Recebe um telefone e retorna seu DDD

    telefone (str or int): Telefone onde os dois primeiros digitios numéricos são o DDD
    """
    pattern = r"[0-9]{2}"
    return "".join(re.findall(pattern, str(telefone)))[:2]


In [ ]:
lista_ddd_19 = list(
    filter(lambda x: True if extrair_ddd(x) == "19" else False, lista_telefones)
)
print(lista_ddd_19)


In [ ]:
filtro_19 = filter(lambda x: True if extrair_ddd(x) == "19" else False, lista_telefones)
for telefone in filtro_19:
    print(telefone)


In [ ]:
map_19 = filter(lambda x: extrair_ddd(x) == "19", lista_telefones)
for i in map_19:
    print(i)


Assim como o `map()`, podemos replicar o comportamento do `filter()` com uma list comprehension. A diferença é que tanto `map()` quanto `filter()` são *lazy*!

In [ ]:
[telefone for telefone in lista_telefones if extrair_ddd(telefone) == "19"]


## Agregando iteráveis com `reduce()`

A função `reduce()` cria a idéia de um `accumulator`. Imagine uma função `sum(x, y)` que calcula a soma de x e y. Podemos utilizar a função `reduce()` (da biblioteca `functools`) para tratar x como o `accumulator` e percorrer o iterador *acumulando* a aplicação da função.

Vamos considerar a lista `[1,4,6,8]` e função soma:

```python
def sum_two_elements(a,b):
    return a+b
```

podemos utilizar a função `reduce()` para somar os elementos de toda a lista:

```python
reduce( sum_two_elements, [1,4,6,8] )
```

Vamos ver o que cada etapa do `reduce()` foi:

```python
a = 0 # accumulator
b = 1 # value
a + b = 1 # so the accumulator receives this cummulative sum

a = 1 # accumulator
b = 4 # value
a + b = 5
...
a = 5 # accumulator
b = 6 # value 
a + b = 11
...
a = 11 # accumulator
b = 8 # value
a + b = 19

return 19
```

In [ ]:
from functools import reduce


### Exemplo 1: Números

In [ ]:
def somar_ab(a, b):
    print(f"a={a}, b={b}")
    return a + b


In [ ]:
lista_numeros = [1, 4, 6, 8]
reduce(somar_ab, lista_numeros)


In [ ]:
def comp_ab(x, y):
    print(f"a={x}, b={y}")
    if x > y:
        return x
    else:
        return y


reduce(comp_ab, [2, 10, 25, 1, -10, 13, 40, 20])


### Exemplo 2: Strings

In [ ]:
lista_letras = ["P", "e", "d", "r", "o"]


In [ ]:
reduce(lambda x, y: x + y, lista_letras)


Um exemplo prático é a utilização do reduce para selecionar o maior string de uma lista:

In [ ]:
lista_nomes = ["Amapá", "Roraima", "Pará", "Piauí", "Maranhão"]
reduce(lambda x, y: x if len(x) > len(y) else y, lista_nomes)


### Vendo as aplicações: `accumulate()`

Nos exemplos acima utilizamos a função `print()` para ver o resultado de cada etapa do `reduce()`. Podemos utilizar a função `accumulate()` para ver o resultado de cada etapa como uma lista:

In [ ]:
from itertools import accumulate


In [ ]:
lista_numeros = [1, 2, 3, 4, 5]
list(accumulate(lista_numeros, lambda x, y: x + y))


In [ ]:
lista_nomes = ["Amapá", "Roraima", "Pará", "Piauí", "Maranhão"]
list(accumulate(lista_nomes, lambda x, y: x if len(x) > len(y) else y))


# Paradigma Funcional no Pandas

Até agora vimos como podemos utilizar as funções `map()`, `filter()` e `reduce()` para aplicar funções à, filtrar e reduzir iteráveis. Embora essas funções sejam extremamente úteis no contexto de dados (principalmente quando lidamos com dados extraídos de APIs ou *webscrapping*), em geral vamos entrar em contato com esses conceitos através do método `.apply()` de DataFrames e Series.

Nos exemplos abaixo, utilizaremos esse método, primeiro sobre `Series`, onde ele se comporta exatamente como o método `.map()`.

In [ ]:
file_address = "http://www.statsci.org/data/general/sleep.txt"
tb_sleep = pd.read_csv(file_address, sep="\t")
tb_sleep.head()


## `.apply()` em Colunas (`Series`)

O método `.apply()`, quando executado a partir de uma `Series` (uma coluna individual) funciona exatamente como o método `.map()`: ele retorna uma nova `Series` (coluna) com o resultado da aplicação de nossa função elemento a elemento na coluna a partir da qual foi invocado:

In [ ]:
tb_sleep["brain_wt_kg"] = tb_sleep["BrainWt"] / 1000
tb_sleep["ratio_brain"] = tb_sleep["brain_wt_kg"] / tb_sleep["BodyWt"]
tb_sleep[["brain_wt_kg", "ratio_brain"]].describe()


Vamos definir uma função para classificar a proporção entre o peso do cérebro e o peso total de cada animal:

In [ ]:
def maior_1p(ratio):
    if ratio > 0.01:
        return "Pesado"
    else:
        return "Leve"


Como podemos aplicar essa função à coluna `ratio_brain`?

In [ ]:
maior_1p(tb_sleep["ratio_brain"])


A aplicação direta não funciona, afinal a função espera um número e `tb_sleep["ratio_brain"]` é uma `Series`!

Vamos utilizar o `.apply()` de `Series` (equivalente ao `.map()`):

In [ ]:
tb_sleep["heavy_brain"] = tb_sleep["ratio_brain"].apply(maior_1p)


In [ ]:
tb_sleep["heavy_brain"].describe()


## `.apply()` em Tabelas (`DataFrame`)

Podemos invocar o método `.apply()` a partir de DataFrames também! Enquanto o método `.apply()` de coluna passa para a função o **valor de cada observação**, o método `.apply()` de tabelas pode passar linhas ou colunas (dependendo do `axis` que escolhermos).

Vamos começar analisando o comportamento desse método utilizando uma função comum do Python: `max`.

### Exemplo 1: Máximo entre colunas

In [ ]:
tb_sleep[["Predation", "Exposure", "Danger"]].apply(max)


O comportamento padrão do `.apply()` de `DataFrames` é aplicar a função sobre as colunas: no caso acima ele retornou o máximo de cada coluna `["Predation", "Exposure", "Danger"]`.

E se quisermos calcular, para cada observação, qual o máximo entre essas três colunas?

In [ ]:
tb_sleep[["Predation", "Exposure", "Danger"]].apply(max, axis=1)


In [ ]:
tb_sleep["max_risco"] = tb_sleep[["Predation", "Exposure", "Danger"]].apply(max, axis=1)
tb_sleep[["Predation", "Exposure", "Danger", "max_risco"]].head()


### Exemplo 2: `.apply()` e funções Lambda

No exemplo acima a função `max` recebeu o iterável `[3, 5, 3]` na primeira linha e calculou o máximo entre eles. No entanto muitas vezes queremos aplicar mais do que uma função simples. 

Por exemplo, podemos calcular a diferença entre o risco máximo e o risco mínimo. Para tanto, utilizaremos uma função lambda:

In [ ]:
tb_sleep["var_risco"] = tb_sleep[["Predation", "Exposure", "Danger"]].apply(
    lambda x: max(x) - min(x), axis=1
)
tb_sleep[["Predation", "Exposure", "Danger", "max_risco", "var_risco"]].head()


Para visualizarmos o que está acontecendo, **precisamos entender o que o método `.apply()` esta passando como `x` para nossa função lambda**!

### Exemplo 3: `.apply()` com argumentos

Nos exemplos acima aplicamos uma função diretamente sobre uma linha (`max` e `min`) - já que estas funções transformam iteráveis em números.

Outra aplicação comum é passar colunas específicas como argumentos particulares de uma função.

In [ ]:
def diff_prop(x, y):
    '''
    Calcula (x-y)/(x+y).
    Arguments:
        x Float
        y Float
    Returns:
        Float
    '''
    return (x-y)/(x+y)

In [ ]:
tb_sleep.apply(
    lambda x: diff_prop(x['NonDreaming'], x['Dreaming']), axis=1
)


No exemplo acima utilizamos o fato que `x` nada mais que uma linha de nosso DataFrame. Podemos utilizar o **desempacotamento de argumentos** para realizar a mesma operação:

In [ ]:
tb_sleep[["NonDreaming", "Dreaming"]].apply(lambda x: diff_prop(*x), axis=1)


In [ ]:
tb_sleep.head()

No caso acima, `x` é um iterável com os valores `NonDreaming` e `Dreaming` - `[6.3, 2.0]` na segunda linha de nosso DataFrame.

Utilizamos `*x` como argumento da função diff_prop, desempacotando esse iterável, passando seu primeiro elemento como o primeiro argumento da função e seu segundo elemento como segundo argumento da função.

Note que nesse caso, a ordem do filtro de colunas no inicio da linha (`tb_sleep[["NonDreaming", "Dreaming"]].apply()`) é determinante no resultado!

In [ ]:
tb_sleep['non_prop'] = tb_sleep[["NonDreaming", "Dreaming"]].apply(lambda x: diff_prop(*x), axis=1)
tb_sleep['dre_prop'] = tb_sleep[["Dreaming", "NonDreaming"]].apply(lambda x: diff_prop(*x), axis=1)
tb_sleep[["NonDreaming", "Dreaming", "non_prop", "dre_prop"]].head()


## Exemplo 4: `.apply()` com strings

In [ ]:
lista_telefones = [
    999571559,
    "2412-0107",
    "99762-1166",
    "4002-8282",
    "3542-1820",
    "9-3561-9525",
    "3333-5802"
]
lista_ddd = [
    19,
    "(11)",
    "(33)",
    "16",
    "29",
    12,
    "(34)",
]
tb_telefone = pd.DataFrame({"ddd" : lista_ddd, "telefone" : lista_telefones})
tb_telefone


In [ ]:
def limpar_telefone(ddd, telefone):
    """
    Extrai o DDD de um telefone
    telefone (str or numeric): telefone onde os dois primeiros digitos numéricos são o DDD
    """
    pattern = r"[0-9]+"
    ddd_limpo = "".join(re.findall(pattern, str(ddd)))
    tel_limpo = "".join(re.findall(pattern, str(telefone)))
    return f"({ddd_limpo}){tel_limpo}"


In [ ]:
limpar_telefone("(19)", 35613870)


In [ ]:
tb_telefone["telefone_completo"] = tb_telefone[["ddd", "telefone"]].apply(lambda x: limpar_telefone(*x), axis = 1)
tb_telefone

## Exemplo 5: `.apply()` com argumentos fixos

Nos exemplos acimas, todos os argumentos das funções aplicadas originavam em colunas do DataFrame. Algumas vezes queremos *fixar* um dos parâmetros de uma função ao aplica-la sobre nosso DataFrame

In [ ]:
def maior_custom(valor, patamar):
    if valor > patamar:
        return "Pesado"
    else:
        return "Leve"


In [ ]:
tb_sleep["ratio_brain"].apply(maior_custom)


In [ ]:
tb_sleep["ratio_brain"].apply(lambda x: maior_custom(x, 0.005))
